# Implementing Random Circuit Sampling on IBM Quantum Devices

## Using IBM Q 5 Yorktown (Automatically Executed, for Data)

Hao Li, Yue Shi, Javad Shabani

New York University

In [1]:
# Import modules to be used in the program

#from qiskit import IBMQ
import numpy as np
from math import *
from qiskit import *
from qiskit.quantum_info import *
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import pauli_error
from qiskit.providers.aer.noise.errors.standard_errors import thermal_relaxation_error
from qiskit.tools.visualization import plot_histogram
from matplotlib import *
import matplotlib.pyplot as plt
import scipy as sp

qiskit.__qiskit_version__

{'qiskit-terra': '0.9.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-aqua': '0.6.0',
 'qiskit': '0.12.0',
 'qiskit-aer': '0.3.0',
 'qiskit-ibmq-provider': '0.3.2'}

In [2]:
# Enable IBM Quantum Experience account, e.g. IBMQ.enable_account('MY_API_TOKEN')
# Do not enable an account more than once in a session

provider = IBMQ.enable_account('')

In [3]:
# Show the backends accessible

provider.backends()
#backend = provider.get_backend('ibmq_rochester')

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]

## For each system, choose the number of qubits in the circuits, the range of the number of layers, the number of circuits to test for each number of layers, and the number of experiments you want to run for each circuit, then the program will be automatically executed, and all the data will be meanwhile saved for further usage

### n-qubit system (n = 3, 4, 5) using ibmqx2

In [4]:
# Choose the parameters for the series of experients to conduct: 
# (1) the number of qubits in the circuits (qubits1), 
# (2) the range of the number of layers of random circuits ([layers1_min, layers1_max]);
# (3) the number of circuits to test for each number of layers (circs1);
# (4) the number of experiments to run for each circuit (runs1)
# Make sure the numbers are correlated

# **If the backend is very busy, it is suggested to split the range of layers or wait for a free, 
#   so as not to let the program run for too long time

backend_1 = provider.get_backend('ibmqx2')

# Parameters for the series of experiments to conduct
qubits1 = [5]
layers1_min = [10]
layers1_max = [15]
circs1 = [20]
runs1 = [1]

# Check if there is anything wrong in the parameters above
qubits_list1 = [3, 4, 5] # The list of the numbers of qubits in circuits available in this program
para_error = False
for n in qubits1:
    if n not in qubits_list1:
        print('Circuits not available for: qubits1[%d]' % qi) # Sequence numbers start from 0!
        para_error = True
if len(layers1_min) != len(qubits1):
    print('Number of parameters mismatched with qubits1: layers1_min')
    para_error = True
if len(layers1_max) != len(qubits1):
    print('Number of parameters mismatched with qubits1: layers1_max')
    para_error = True
if len(circs1) != len(qubits1):
    print('Number of parameters mismatched with qubits1: circs1')
    para_error = True
if len(runs1) != len(qubits1):
    print('Number of parameters mismatched with qubits1: runs1')
    para_error = True
if para_error == False:
    print('Parameters confirmed!')

Parameters confirmed!


In [5]:
# Automatically execute the program to perform all the experiments described above and save data

for qi in range(len(qubits1)):
    # The number of qubits used in the circuits, and the dimension of the Hilbert space
    n = qubits1[qi]
    N = 2**n
    for layers1 in range(layers1_min[qi], layers1_max[qi] + 1):
        for n_circ1 in range(1, circs1[qi] + 1):
            # 5 qubit circuits initialization (using ibmqx2)
            # The number of qubits registered
            n_reg1 = n
            q1 = QuantumRegister(n_reg1)
            c1 = ClassicalRegister(n_reg1)
            circ1 = QuantumCircuit(q1, c1)
            # Generate 5-qubit random circuits of a number of layers
            for k in range(0, layers1):
                # For every qubit, add a random unitary u3 gate
                for i in range(0, n):    
                    u = quantum_info.random_unitary(2).data
                    Theta = 2 * np.arccos(abs(u[0][0]))
                    Phi = np.angle(u[1][0] / (u[0][0] * np.sqrt(1 - abs(u[0][0])**2)))
                    Lambda =  np.angle(- u[0][1] / (u[0][0] * np.sqrt(1 - abs(u[0][0])**2)))
                    Phi += 2 * np.pi * int(- np.sign(Phi) + 0.5)
                    Lambda += 2 * np.pi * int(- np.sign(Lambda) + 0.5)
                    circ1.u3(Theta, Phi, Lambda, q1[i])
                if n == 3:
                    # Apply CNOT on a pair of qubits in random direction
                    j = np.random.randint(0, n)
                    l1 = j
                    while l1 == j:
                        l1 = np.random.randint(0, n)
                    circ1.cx(q1[j], q1[l1])
                else:
                    # Apply CNOT on qubit 0 & 1 in random direction
                    l1 = np.random.randint(0, 2)
                    if l1 == 0:
                        circ1.cx(q1[0], q1[1])
                    else:
                        circ1.cx(q1[1], q1[0])
                    if n == 5:
                        # Apply CNOT on qubit 3 & 4 in random direction
                        l2 = np.random.randint(0, 2)
                        if l2 == 0:
                            circ1.cx(q1[3], q1[4])
                        else:
                            circ1.cx(q1[4], q1[3])
                    # Apply CNOT on qubit 2 & a random qubit in random direction
                    j = 2
                    while j == 2:
                        j = np.random.randint(0, n)
                    l3 = np.random.randint(0, 2)
                    if l3 == 0:
                        circ1.cx(q1[2], q1[j])
                    else:
                        circ1.cx(q1[j], q1[2])
            
            # Run it on statevector_simulator to work out the state amplitudes
            circuit = circ1
            simulator = Aer.get_backend('statevector_simulator')
            job1 = execute(circuit, simulator)
            result1 = job1.result()
            jobID1 = job1.job_id()
            # Get the statevector from result1()
            statevector1 = result1.get_statevector(circuit)
            # Get the probabilities (state1 with data cleaning order) and sort (state1_sort)
            state1 = []
            stvec1 = []
            for i in range(0, 2**n_reg1):
                if abs(statevector1[i]) != 0:
                    state1.append(np.abs(statevector1[i])**2)
                    stvec1.append(statevector1[i])
            state1 = np.array(state1)
            state1_sort = - np.sort(- state1)
            # Save the statevector simulation results to a file (in data cleaning order)
            stvec1 = np.array(stvec1)
            data1 = np.column_stack((stvec1, state1))
            filename = "%d-qubit_%s_stvec_L%d_C%d_%s.csv" % (n, backend_1, layers1, n_circ1, 
                                                             jobID1)
            np.savetxt(filename, data1, delimiter = ',')
            # Ideal (discrete) Porter-Thomas distribution
            f = []
            for i in range(0, N):
                if i == 0:
                    f.append((log(N / (i + 1)) + 1) / N)
                else:
                    f.append((log(N / (i + 1)) + 1 - i * log ((i + 1) / i)) / N)
            # XEB fidelities of statevector simulation (original and simplified)
            S_inc_ptd = 0
            S_exp_ptd = 0
            S_ptd = 0
            for i in range(0, N):
                S_inc_ptd += - log(f[i]) / N
                S_exp_ptd += - log(f[i]) * state1_sort[i]
                S_ptd += - log(f[i]) * f[i]
            F_XEB_5_stvec = (S_inc_ptd - S_exp_ptd) / (S_inc_ptd - S_ptd)
            F_XEB_sim_5_stvec = log(N) + np.euler_gamma - S_exp_ptd
            # Linear XEB (LXEB) fidelities of statevecror simulation (original and simplified)
            SL_inc_ptd = 0
            SL_exp_ptd = 0
            SL_ptd = 0
            for i in range(0, N):
                SL_inc_ptd += (1 - f[i]) / N
                SL_exp_ptd += (1 - f[i]) * state1_sort[i]
                SL_ptd += (1 - f[i]) * f[i]
            F_LXEB_5_stvec = (SL_inc_ptd - SL_exp_ptd) / (SL_inc_ptd - SL_ptd)
            F_LXEB_sim_5_stvec = 2**n * (1 - SL_exp_ptd) - 1
            
            # Save the circuit (no measurement) for statevector simulation as an OpenQasm file
            circ1.qasm()
            file = open("%d_qubit_circuit_%s_no_mea_L%d_C%d.qasm" % 
                        (n, backend_1, layers1, n_circ1), 'w')
            file.write(circ1.qasm())
            file.close()
            
            # Apply measurement gates to the circuit, so that you can run it on IBM device
            for i in range(0, n):
                circ1.measure(q1[i], c1[i])
            
            # Save the circuit as an OpenQasm file
            circ1.qasm()
            file = open("%d_qubit_circuit_%s_L%d_C%d.qasm" % 
                        (n, backend_1, layers1, n_circ1), 'w')
            file.write(circ1.qasm())
            file.close()
        
            # Run it on actual IBM system (ibmqx2)
            counts_1_total = []
            prob_1_total = []
            jobID_1 = []
            for run in range(1, runs1[qi] + 1):
                shots_1 = 8192
                backend_1 = provider.get_backend('ibmqx2')
                job_1 = execute(circ1, backend_1, shots = shots_1)
                result_1 = job_1.result().get_counts(circ1)
                jobID_1.append(job_1.job_id())
                # Data cleaning states(st_1), experimental counts(counts_1) & 
                # probabilities(prob_1)
                st = []
                counts = []
                prob = []
                for i in range(0, 2**n_reg1):
                    s = '0' * n_reg1
                    bin_i = np.binary_repr(i)
                    s_i = s[0: n_reg1 - len(bin_i)] + bin_i
                    st.append(s_i)
                    prob.append(np.abs(statevector1[i])**2)
                    try:
                        counts.append(result_1[s_i])
                    except:
                        counts.append(0)
                st_1 = []
                counts_1 = []
                prob_1 = []
                for i in range(len(prob)):
                    if prob[i] != 0:
                        st_1.append(st[i])
                        counts_1.append(counts[i])
                        prob_1.append(counts[i] / shots_1) 
                if run == 1:
                    counts_1_total = np.array(counts_1)
                else:
                    counts_1_total = np.array(counts_1_total) + np.array(counts_1)
                # Save the experimental results to files separately
                data_1 = np.column_stack((counts_1, prob_1))
                filename = "%d-qubit_%s_L%d_C%d_R%d_%s.csv" % (n, backend_1, layers1, n_circ1, 
                                                               run, jobID_1[run - 1]) 
                np.savetxt(filename, data_1, delimiter = ',')
            # Save the total counts and probabilities of all these results
            prob_1_total = np.array(counts_1_total) / (runs1[qi] * shots_1)
            data_1_total = np.column_stack((counts_1_total, prob_1_total))
            filename = "%d_qubit_%s_L%d_C%d_R%d_total.csv" % (n, backend_1, layers1, n_circ1, 
                                                              runs1[qi])
            np.savetxt(filename, data_1_total, delimiter = ',')
            
            # Run it on Qasm simulator to simulate results on classical computer
            shots_2 = 2**18
            simulator = Aer.get_backend('qasm_simulator')
            math_dimain_error = True
            while math_dimain_error == True:
                math_dimain_error = False
                job_2 = execute(circ1, simulator, shots = shots_2)
                result_2 = job_2.result().get_counts(circ1)
                jobID_2 = job_2.job_id()
                # Data cleaning states(st_2), simulated counts(counts_2) & probabilities(prob_2)
                st = []
                counts = []
                prob = []
                for i in range(0, 2**n_reg1):
                    s = '0' * n_reg1
                    bin_i = np.binary_repr(i)
                    s_i = s[0: n_reg1 - len(bin_i)] + bin_i
                    st.append(s_i)
                    prob.append(np.abs(statevector1[i])**2)
                    try:
                        counts.append(result_2[s_i])
                    except:
                        counts.append(0)
                st_2 = []
                counts_2 = []
                prob_2 = []
                for i in range(len(prob)):
                    if prob[i] != 0:
                        st_2.append(st[i])
                        if counts[i] == 0:
                            math_domain_error = True
                        counts_2.append(counts[i])
                        prob_2.append(counts[i] / shots_2)     
            # Save the qasm simulation results to a file (in data cleaning order)
            data_2 = np.column_stack((counts_2, prob_2))
            filename = "%d-qubit_%s_qasm_L%d_C%d_%s.csv" % (n, backend_1, layers1, n_circ1, 
                                                            jobID_2) 
            np.savetxt(filename, data_2, delimiter = ',')
            # Sort the Qasm simulation and experimental results by Qasm simulation (descending)
            prob_1_total_sort = []
            prob_2_sort = []
            prob_1_total_sort = np.array(prob_1_total)
            prob_2_sort = - np.array(prob_2)
            prob_2_sort, prob_1_total_sort = zip(*sorted(zip(prob_2_sort, prob_1_total_sort)))
            prob_2_sort = - np.array(prob_2_sort)
            # XEB fidelities of Qasm simulation (original and simplified)
            S_inc_ptd = 0
            S_exp_ptd = 0
            S_ptd = 0
            for i in range(0, N):
                S_inc_ptd += - log(f[i]) / N
                S_exp_ptd += - log(f[i]) * prob_2_sort[i]
                S_ptd += - log(f[i]) * f[i]
            F_XEB_5_qasm = (S_inc_ptd - S_exp_ptd) / (S_inc_ptd - S_ptd)
            F_XEB_sim_5_qasm = log(N) + np.euler_gamma - S_exp_ptd
            # LXEB fidelities of Qasm simulation (original and simplified)
            SL_inc_ptd = 0
            SL_exp_ptd = 0
            SL_ptd = 0
            for i in range(0, N):
                SL_inc_ptd += (1 - f[i]) / N
                SL_exp_ptd += (1 - f[i]) * prob_2_sort[i]
                SL_ptd += (1 - f[i]) * f[i]
            F_LXEB_5_qasm = (SL_inc_ptd - SL_exp_ptd) / (SL_inc_ptd - SL_ptd)
            F_LXEB_sim_5_qasm = N * (1 - SL_exp_ptd) - 1
            # XEB fidelities of experimental results (original and simplified)
            S_inc_exp = 0
            S_mea_exp = 0
            S_exp = 0
            for i in range(0, N):
                S_inc_exp += - log(prob_2[i]) / N
                S_mea_exp += - log(prob_2[i]) * prob_1_total[i]
                S_exp += - log(prob_2[i]) * prob_2[i]
            F_XEB_5 = (S_inc_exp - S_mea_exp) / (S_inc_exp - S_exp)
            F_XEB_sim_5 = log(N) + np.euler_gamma - S_mea_exp
            # LXEB fidelities of experimental results (original and simplified)
            SL_inc_exp = 0
            SL_mea_exp = 0
            SL_exp = 0
            for i in range(0, N):
                SL_inc_exp += (1 - prob_2[i]) / N
                SL_mea_exp += (1 - prob_2[i]) * prob_1_total[i]
                SL_exp += (1 - prob_2[i]) * prob_2[i]
            F_LXEB_5 = (SL_inc_exp - SL_mea_exp) / (SL_inc_exp - SL_exp)
            F_LXEB_sim_5 = N * (1 - SL_mea_exp) - 1
            # k-th order XEB fidelities (k = 2, 3, 4, 5, 6, and inverse) (original)
            F_kXEB_5 = []
            F_kXEB_sim_5 = []
            for k in range(2, 7):
                Sk_inc_exp = 0
                Sk_mea_exp = 0
                Sk_exp = 0
                Pk_mea_exp = 0
                for i in range(0, N):
                    Sk_inc_exp += (1 - prob_2[i]**k) / N
                    Sk_mea_exp += (1 - prob_2[i]**k) * prob_1_total[i]
                    Sk_exp += (1 - prob_2[i]**k) * prob_2[i]
                    Pk_mea_exp += (prob_2[i]**k) * prob_1_total[i]
                F_kXEB_5.append((Sk_inc_exp - Sk_mea_exp) / (Sk_inc_exp - Sk_exp))
                F_kXEB_sim_5.append((Pk_mea_exp * N**k / gamma(k + 1) - 1) / k)
            F_1kXEB_5 = []
            F_1kXEB_sim_5 = []
            for k in range(2, 7):
                S1k_inc_exp = 0
                S1k_mea_exp = 0
                S1k_exp = 0
                P1k_mea_exp = 0
                for i in range(0, N):
                    S1k_inc_exp += (1 - prob_2[i]**(1 / k)) / N
                    S1k_mea_exp += (1 - prob_2[i]**(1 / k)) * prob_1_total[i]
                    S1k_exp += (1 - prob_2[i]**(1 / k)) * prob_2[i]
                    P1k_mea_exp += (prob_2[i]**(1 / k)) * prob_1_total[i]
                F_1kXEB_5.append((S1k_inc_exp - S1k_mea_exp) / (S1k_inc_exp - S1k_exp))
                F_1kXEB_sim_5.append((P1k_mea_exp * N**(1 / k) / gamma(1 / k + 1) - 1) * k)
            
            # Build noise model for noise simulation
            basis_gates = ['u1', 'u2', 'u3', 'cx', 'id']
            noise_model1 = NoiseModel(basis_gates)
            p_ui = 0.001 # Single qubit unitary gate Pauli error rate
            p_id = 0.001 # Identical gate Pauli error rate
            p_cx0 = 0.005 # CNOT Pauli error rate 'XX'
            p_cx1 = 0.02 # CNOT Pauli error rate 'XI'
            p_cx2 = 0.02 # CNOT Pauli error rate 'IX'
            p1given0 = 0.03 # Readout error to get |1> given |0>
            p0given1 = 0.03 # Readout error to get |0> given |1>
            T1 = 57.6 # Qubit population thermal relaxation time (in microseconds)
            T2 = 58.4 # Qubit phase thermal relaxation time (in microseconds)
            t1 = 0.05 # Single qubit gate time (in microseconds)
            t2 = 0.1 # Two qubit gate time (in microseconds)
            esp = 0.001 # Excited state population at equilibrium (default: |0>)
            p_mea = 0.01 # Measurement Pauli error rate
            p_res = 0.01 # Reset Pauli error rate
            error_ui = pauli_error([('X', p_ui), ('I', 1 - p_ui)])
            error_id = pauli_error([('X', p_id), ('I', 1 - p_id)])
            error_cx = pauli_error([('XX', p_cx0), ('XI', p_cx1), 
                                    ('IX', p_cx2), ('II', 1 - p_cx0 - p_cx1 - p_cx2)])
            error_ro = [[1 - p1given0, p1given0], [p0given1, 1 - p0given1]]
            error_tr1 = thermal_relaxation_error(T1, T2, t1, esp)
            error_tr2 = (thermal_relaxation_error(T1, T2, t2, esp).
                         expand(thermal_relaxation_error(T1, T2, t2, esp)))
            error_mea = pauli_error([('X', p_mea), ('I', 1- p_mea)])
            error_res = pauli_error([('X', p_res), ('I', 1 - p_res)])
            # All qubit error for each gate can only be applied once
            #noise_model1.add_all_qubit_quantum_error(error_ui, ['u1', 'u2', 'u3'])
            #noise_model1.add_all_qubit_quantum_error(error_id, 'id')
            noise_model1.add_all_qubit_quantum_error(error_cx, 'cx')
            noise_model1.add_all_qubit_readout_error(error_ro)
            noise_model1.add_all_qubit_quantum_error(error_tr1, ['u1', 'u2', 'u3'])
            noise_model1.add_all_qubit_quantum_error(error_tr1, 'id')
            #noise_model1.add_all_qubit_quantum_error(error_tr2, 'cx')
            noise_model1.add_all_qubit_quantum_error(error_mea, 'measure')
            noise_model1.add_all_qubit_quantum_error(error_res, 'reset')
            #noise_model1.add_quantum_error(error, 'reset', [1])
            #noise_model1.add_nonlocal_quantum_error(error, 'reset', [0], [1])
            # Get coupling map from backend
            backend_1 = provider.get_backend('ibmqx2')
            coupling_map = backend_1.configuration().coupling_map
            # Run it on Qasm simulator to simulate results with noise on classical computer
            shots_3 = 2**18
            simulator = Aer.get_backend('qasm_simulator')
            job_3 = execute(circ1, simulator, noise_model = noise_model1, 
                            coupling_map = coupling_map, shots = shots_3)
            result_3 = job_3.result().get_counts(circ1)
            jobID_3 = job_3.job_id()
            # Data cleaning states(st_3), simulated counts(counts_3) & probabilities(prob_3)
            st = []
            counts = []
            prob = []
            for i in range(0, 2**n_reg1):
                s = '0' * n_reg1
                bin_i = np.binary_repr(i)
                s_i = s[0: n_reg1 - len(bin_i)] + bin_i
                st.append(s_i)
                prob.append(np.abs(statevector1[i])**2)
                try:
                    counts.append(result_3[s_i])
                except:
                    counts.append(0) 
            st_3 = []
            counts_3 = []
            prob_3 = []
            for i in range(len(prob)):
                if prob[i] != 0:
                    st_3.append(st[i])
                    counts_3.append(counts[i])
                    prob_3.append(counts[i] / shots_3)
            # Save the Qasm noise simulation results to a file (in data cleaning order)
            data_3 = np.column_stack((counts_3, prob_3))
            filename = "%d-qubit_%s_qasm_noise_L%d_C%d_%s.csv" % (n, backend_1, layers1, n_circ1, 
                                                                  jobID_3)
            np.savetxt(filename, data_3, delimiter = ',')
            # Sort the Qasm and noise simulation results by Qasm simulation results (descending)
            prob_3_sort = []
            prob_2_sort = []
            prob_3_sort = np.array(prob_3)
            prob_2_sort = - np.array(prob_2)
            prob_2_sort, prob_3_sort = zip(*sorted(zip(prob_2_sort, prob_3_sort)))
            prob_2_sort = - np.array(prob_2_sort)
            # XEB fidelities of Qasm noise simulation (original and simplified) 
            S_inc_ptd = 0
            S_exp_ptd = 0
            S_ptd = 0
            for i in range(0, N):
                S_inc_ptd += - log(f[i]) / N
                S_exp_ptd += - log(f[i]) * prob_3_sort[i]
                S_ptd += - log(f[i]) * f[i]
            F_XEB_5_qasm_noise = (S_inc_ptd - S_exp_ptd) / (S_inc_ptd - S_ptd)
            F_XEB_sim_5_qasm_noise = log(N) + np.euler_gamma - S_exp_ptd
            # Linear XEB fidelities of Qasm noise simulation (original and simplified)
            SL_inc_exp = 0
            SL_mea_exp = 0
            SL_exp = 0
            for i in range(0, N):
                SL_inc_exp += (1 - f[i]) / N
                SL_mea_exp += (1 - f[i]) * prob_3_sort[i]
                SL_exp += (1 - f[i]) * f[i]
            F_LXEB_5_qasm_noise = (SL_inc_exp - SL_mea_exp) / (SL_inc_exp - SL_exp)
            F_LXEB_sim_5_qasm_noise = N * (1 - SL_mea_exp) - 1
            # Save all the calculated fidelities ((noise) simulation, and experiment)
            filename = "%d-qubit_%s_fidelities_L%d_C%d.csv" % (n, backend_1, layers1, n_circ1)
            file1 = open(filename, "w")
            file1.write("%s\n" % backend_1.status())
            file1.write("Number of qubits in the circuit, %d\n" % n)
            file1.write("Number of layers of circuits, %d\n" % layers1)
            file1.write("Circuit sequence number, %d\n" % n_circ1)
            file1.write("Depth of the circuit, %d\n" % circ1.depth())
            file1.write("Number of runs (8192 times each), %d\n\n" % runs1[qi])
            file1.write("Job ID of simulations and experiments\n")
            file1.write("Statevector simulation, %s\n" % jobID1)
            file1.write("Qasm simulation, %s\n" % jobID_2)
            file1.write("Qasm noise simulation, %s\n" % jobID_3)
            for run in range(1, runs1[qi] + 1):
                file1.write("%s experiment %d, %s\n" % (backend_1, run, jobID_1[run - 1]))
            file1.write("\n")
            file1.write("Fidelities (original and simplified) of simulations and experiment,")
            file1.write("Origial idelities, Simplified fidelities, k for k'th order fidelities\n")
            file1.write("F_XEB(_sim)_statevector, %.17f, %.17f\n" % 
                        (F_XEB_5_stvec, F_XEB_sim_5_stvec))
            file1.write("F_LXEB(_sim)_statevector, %.17f, %.17f\n" % 
                        (F_LXEB_5_stvec, F_LXEB_sim_5_stvec))
            file1.write("F_XEB(_sim)_qasm, %.17f, %.17f\n" % (F_XEB_5_qasm, F_XEB_sim_5_qasm))
            file1.write("F_LXEB(_sim)_qasm, %.17f, %.17f\n" % (F_LXEB_5_qasm, F_LXEB_sim_5_qasm))
            file1.write("F_XEB(_sim)_qasm_noise, %.17f, %.17f\n" % 
                        (F_XEB_5_qasm_noise, F_XEB_sim_5_qasm_noise))
            file1.write("F_LXEB(_sim)_qasm_noise, %.17f, %.17f\n" % 
                        (F_LXEB_5_qasm_noise, F_LXEB_sim_5_qasm_noise))
            file1.write("F_XEB(_sim), %.17f, %.17f\n" % (F_XEB_5, F_XEB_sim_5))
            file1.write("F_LXEB(_sim), %.17f, %.17f, 1\n" % (F_LXEB_5, F_LXEB_sim_5))
            for k in range (2, 7):
                file1.write("F_kXEB(_sim), %.17f, %.17f, %d\n" % 
                            (F_kXEB_5[k - 2], F_kXEB_sim_5[k - 2], k))
            for k in range (2, 7):
                file1.write("F_kXEB(_sim), %.17f, %.17f, %.17f, 1/%d\n" % 
                            (F_1kXEB_5[k - 2], F_1kXEB_sim_5[k - 2], (1 / k), k))
            file1.close()
            
            print("Qubits: %d, Layers: %d, Circuits: %d" % (n, layers1, n_circ1))

print("Finished!")

Qubits: 5, Layers: 10, Circuits: 1
Qubits: 5, Layers: 10, Circuits: 2
Qubits: 5, Layers: 10, Circuits: 3
Qubits: 5, Layers: 10, Circuits: 4
Qubits: 5, Layers: 10, Circuits: 5
Qubits: 5, Layers: 10, Circuits: 6
Qubits: 5, Layers: 10, Circuits: 7
Qubits: 5, Layers: 10, Circuits: 8
Qubits: 5, Layers: 10, Circuits: 9
Qubits: 5, Layers: 10, Circuits: 10
Qubits: 5, Layers: 10, Circuits: 11
Qubits: 5, Layers: 10, Circuits: 12
Qubits: 5, Layers: 10, Circuits: 13
Qubits: 5, Layers: 10, Circuits: 14
Qubits: 5, Layers: 10, Circuits: 15
Qubits: 5, Layers: 10, Circuits: 16
Qubits: 5, Layers: 10, Circuits: 17
Qubits: 5, Layers: 10, Circuits: 18
Qubits: 5, Layers: 10, Circuits: 19
Qubits: 5, Layers: 10, Circuits: 20
Qubits: 5, Layers: 11, Circuits: 1
Qubits: 5, Layers: 11, Circuits: 2
Qubits: 5, Layers: 11, Circuits: 3
Qubits: 5, Layers: 11, Circuits: 4
Qubits: 5, Layers: 11, Circuits: 5
Qubits: 5, Layers: 11, Circuits: 6
Qubits: 5, Layers: 11, Circuits: 7
Qubits: 5, Layers: 11, Circuits: 8
Qubits: 5